In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Canavan_Disease"
cohort = "GSE41445"

# Input paths
in_trait_dir = "../../input/GEO/Canavan_Disease"
in_cohort_dir = "../../input/GEO/Canavan_Disease/GSE41445"

# Output paths
out_data_file = "../../output/preprocess/Canavan_Disease/GSE41445.csv"
out_gene_data_file = "../../output/preprocess/Canavan_Disease/gene_data/GSE41445.csv"
out_clinical_data_file = "../../output/preprocess/Canavan_Disease/clinical_data/GSE41445.csv"
json_path = "../../output/preprocess/Canavan_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Basal gene expression data of 21 cell lines (18 cancer and 3 non-tumorigenic)"
!Series_summary	"We profiled basal gene expressed levels of 21 cell lines (18 cancer and 3 non-tumorigenic) using Affymetrix HG-U133_plus2 GeneChip microarrays."
!Series_summary	"Goal of the experiment was to benchmark a number of algorithms for biomarker detection all of which utilize gene expression data."
!Series_overall_design	"Cell lines were cultivated in appropriate media according to supplier recommendations. Goal was to acquire a basal gene expression profile of 21 common cell lines."
Sample Characteristics Dictionary:
{0: ['gender: male', 'gender: female'], 1: ['organ: Kidney', 'organ: Lung', 'organ: Colon', 'organ: Prostate', 'organ: Skin', 'organ: Cervix', 'organ: Breast', 'organ: Pancreas', 'organ: Ovary', 'organ: Bone'], 2: ['disease: renal cell adenocarcinoma', 'disease: non-small cell carcinoma', 'disease: colorectal adenocarcinoma', 'disease: moderately

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# Analyze the data
# 1. Gene Expression Data Availability
# Given the background info: "Affymetrix HG-U133_plus2 GeneChip microarrays" indicates gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Canavan Disease), look for it in disease field
trait_row = 2  # Disease information is in row 2
# Age is not available in the sample characteristics
age_row = None
# Gender is available
gender_row = 0

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait information to binary: 1 for Canavan disease, 0 for other"""
    if value is None or ':' not in value:
        return None
    
    disease_part = value.split(':', 1)[1].strip().lower()
    # Check for Canavan disease, including partial mentions or indications
    if 'canavan disease' in disease_part or 'aspartoacylase deficiency' in disease_part:
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age to continuous value - not used as age data is unavailable"""
    if value is None or ':' not in value:
        return None
    
    # If we had age data, we would extract and convert to float
    age_part = value.split(':', 1)[1].strip()
    try:
        return float(age_part)
    except:
        return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male"""
    if value is None or ':' not in value:
        return None
    
    gender_part = value.split(':', 1)[1].strip().lower()
    if 'female' in gender_part:
        return 0
    elif 'male' in gender_part:
        return 1
    else:
        return None

# 3. Save Metadata - Initial Filtering
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                              is_gene_available=is_gene_available, 
                              is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# Properly create the clinical data DataFrame from the sample characteristics dictionary
sample_chars_dict = {0: ['gender: male', 'gender: female'], 
                     1: ['organ: Kidney', 'organ: Lung', 'organ: Colon', 'organ: Prostate', 'organ: Skin', 'organ: Cervix', 'organ: Breast', 'organ: Pancreas', 'organ: Ovary', 'organ: Bone'], 
                     2: ['disease: renal cell adenocarcinoma', 'disease: non-small cell carcinoma', 'disease: colorectal adenocarcinoma', 'disease: moderately differentiated adenocarcinoma', 'disease: non-tumorigenic; spontaneously immortilized keratinocytes', 'disease: colorectal carcinoma', 'disease: adenocarcinoma', 'disease: non-tumorigenic; aspartoacylase deficiency; possible Canavan disease', 'disease: invasive ductal carcinoma; inflammatory skin metastasis', 'disease: non-tumorigenic; Fibrocystic Breast disease', 'disease: intraductal adenocarcinoma', 'disease: carcinoma', 'disease: large cell carcinoma', 'disease: Gleason Grade 4; adenocarcinoma', 'disease: malignant melanoma', "disease: Dukes' type B; colorectal adenocarcinoma", 'disease: ductal carcinoma', 'disease: osteosarcoma'], 
                     3: ['morphology: epithelial', 'morphology: fibroblast', 'morphology: attached epithelial with floating rounded cells', 'morphology: polygonal'], 
                     4: ['is_metastasis: no', 'is_metastasis: yes'], 
                     5: ['metastatic_site: n/a', 'metastatic_site: brain', 'metastatic_site: pleural fluid', 'metastatic_site: bone', 'metastatic_site: ascites'], 
                     6: ['in_vivo_growth: yes', 'in_vivo_growth: not tested'], 
                     7: ['cell line: 786-O', 'cell line: A549', 'cell line: Caco-2', 'cell line: DU 145', 'cell line: HaCaT', 'cell line: HCT 116', 'cell line: HeLa', 'cell line: Hs68', 'cell line: HT-29', 'cell line: KPL-4', 'cell line: MCF 10A', 'cell line: MCF7', 'cell line: MDA-MB-231', 'cell line: Mia PaCa-2', 'cell line: NCI-H460', 'cell line: PC-3', 'cell line: SK-MEL-28', 'cell line: SK-OV-3', 'cell line: SW480', 'cell line: T47D', 'cell line: U-2 OS'], 
                     8: ['atcc_number: CRL-1932', 'atcc_number: CCL-185', 'atcc_number: HTB-37', 'atcc_number: HTB-81', 'atcc_number: n/a (PMID: 2450098)', 'atcc_number: CCL-247', 'atcc_number: CCL-2', 'atcc_number: CRL-1635', 'atcc_number: HTB-38', 'atcc_number: n/a (PMID: 10070858)', 'atcc_number: CRL-10317', 'atcc_number: HTB-22', 'atcc_number: HTB-26', 'atcc_number: CRL-1420', 'atcc_number: HTB-177', 'atcc_number: CRL-1435', 'atcc_number: HTB-72', 'atcc_number: HTB-77', 'atcc_number: CCL-228', 'atcc_number: HTB-133', 'atcc_number: HTB-96']}

clinical_data = pd.DataFrame.from_dict(sample_chars_dict, orient='index')

# If trait_row is not None, extract clinical features
if trait_row is not None:
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Preview of selected clinical features:
{0: [0.0, 1.0], 1: [0.0, 0.0], 2: [0.0, nan], 3: [0.0, nan], 4: [0.0, nan], 5: [0.0, nan], 6: [0.0, nan], 7: [1.0, nan], 8: [0.0, nan], 9: [0.0, nan], 10: [0.0, nan], 11: [0.0, nan], 12: [0.0, nan], 13: [0.0, nan], 14: [0.0, nan], 15: [0.0, nan], 16: [0.0, nan], 17: [0.0, nan], 18: [nan, nan], 19: [nan, nan], 20: [nan, nan]}
Clinical data saved to: ../../output/preprocess/Canavan_Disease/clinical_data/GSE41445.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers provided in the output above, I can see they follow the Affymetrix probe ID format
# (e.g., "1007_s_at", "1053_at") rather than standard human gene symbols.
# These are probe IDs from an Affymetrix microarray that need to be mapped to human gene symbols

# These IDs need to be mapped to standard gene symbols for proper analysis
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the previews, we need to map the probe IDs to gene symbols
# The 'ID' column in gene_annotation matches the index of gene_data
# The 'Gene Symbol' column contains the gene symbols we need
probe_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get a gene mapping dataframe by extracting the relevant columns
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# This function handles the many-to-many relation between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print shape of the resulting gene expression dataframe
print(f"Shape of mapped gene expression data: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])


Shape of mapped gene expression data: (21278, 63)
First few gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Reload the clinical data to understand what's happening with the trait values
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print("Clinical data shape:", clinical_df.shape)
print("Clinical data preview:")
print(clinical_df.head())

# The problem is likely that the clinical data extraction didn't successfully identify Canavan disease samples
# Reload the original clinical data to see the actual values
background_info, clinical_data = get_background_and_clinical_data(matrix_file)
unique_values_dict = get_unique_values_by_row(clinical_data)
print("\nDisease values (row 2):")
print(unique_values_dict[2])

# Update the trait conversion function to better identify Canavan disease cases
def improved_convert_trait(value):
    """Convert trait information to binary: 1 for Canavan disease, 0 for other"""
    if value is None or ':' not in value:
        return None
    
    disease_part = value.split(':', 1)[1].strip().lower()
    # Look for clearer indicators of Canavan disease
    if 'canavan disease' in disease_part or 'aspartoacylase deficiency' in disease_part:
        return 1
    else:
        return 0

# Extract clinical features using updated function
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=2,  # Disease information is in row 2
    convert_trait=improved_convert_trait,
    gender_row=0,
    convert_gender=convert_gender
)

# Save updated clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"\nUpdated clinical data saved to {out_clinical_data_file}")

# Now link the updated clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains 21 cell lines including one with aspartoacylase deficiency (Canavan disease)"
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Canavan_Disease/gene_data/GSE41445.csv
Clinical data shape: (2, 21)
Clinical data preview:
                   0    1    2    3    4    5    6    7    8    9  ...   11  \
Canavan_Disease  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0   
Gender           1.0  0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   

                  12   13   14   15   16   17  18  19  20  
Canavan_Disease  0.0  0.0  0.0  0.0  0.0  0.0 NaN NaN NaN  
Gender           NaN  NaN  NaN  NaN  NaN  NaN NaN NaN NaN  

[2 rows x 21 columns]

Disease values (row 2):
['disease: renal cell adenocarcinoma', 'disease: non-small cell carcinoma', 'disease: colorectal adenocarcinoma', 'disease: moderately differentiated adenocarcinoma', 'disease: non-tumorigenic; spontaneously immortilized keratinocytes', 'disease: colorectal carcinoma', 'disease: adenocarcinoma', 'disease: non-tumorigenic; aspartoacylase deficiency; possible Canavan disease', 'disease: i

For the feature 'Canavan_Disease', the least common label is '1.0' with 3 occurrences. This represents 4.76% of the dataset.
The distribution of the feature 'Canavan_Disease' in this dataset is severely biased.

For the feature 'Gender', the least common label is '1.0' with 27 occurrences. This represents 42.86% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Canavan_Disease/cohort_info.json
Data was determined to be unusable and was not saved
